This notebook explores the patterns in the grid frequency and frequency control power. 

In [111]:
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.express as px

pd.options.display.float_format = '{:.1f}'.format

## BESS data
This is the data captured where the battery system connects to the grid.

Refer to https://publications.rwth-aachen.de/record/985923/files/Report_04-2023.pdf for details.

Description of the data:

| Variable | Description | Unit |
| ---- | ---- | ---- |
| DateAndTime | Date and Time | UTC Timezone ('yyyy-MM-dd HH:mm:ss')|
| M5BAT_P | Active power of M5BAT measured at the network node | kW (- = charging; + = discharging) |
| M5BAT_Q | Reactive power of M5BAT measured at the network node | kVAr |
| Grid_frequency | Grid frequency measured at the network node | mHz |
| Temperature | Ambient temperature at M5BAT site | 0.1°C |
| FCR_activated | Activation signal for FCR | True = FCR activated |
| FCR_P | Active Power for FCR (calculation) | kW (- = charging; + = discharging) | 
| FCR_control | Control band for FCR | kW |
| SPA_ask_P | Request for active power for setpoint adjustment | kW (- = charging; + = discharging) |
| SPA_exec_P | Active power for setpoint adjustment | kW (- = charging; + = discharging) |
| SOC | State of Charge for M5BAT (calculated) | %|
| Interpolated | Interpolation signal for data evaluation | True = Value linear interpolated|

In [178]:
bess = pd.read_csv('BESS.csv', sep=';', parse_dates=['DateAndTime'], index_col='DateAndTime')
# Shift the times into the correct timezone (CET/CEST) but then convert back to tz-less datetimes to avoid hassles
bess.index = bess.index.tz_localize('utc').tz_convert('Europe/Berlin')
bess.index = bess.index.tz_localize(None)

bess['grid_freq_centered'] = bess.Grid_frequency - 50000
bess['grid_freq_sign'] = np.sign(bess.grid_freq_centered)


In [184]:
# Theres a section of the data where the grid frquency is erroneously zero; we remove it.
nz = bess.loc[bess.Grid_frequency >0].copy()
nz.FCR_P.describe()

count   2589767.0
mean         -4.4
std         372.9
min       -1815.0
25%        -240.0
50%           0.0
75%         240.0
max        2295.0
Name: FCR_P, dtype: float64

In [185]:
# Start by taking a look at a summary of the (shifted) grid frequency

tp = nz.resample('1h').grid_freq_centered.median()
px.line(tp)

### Does the grid frequency follow any typical daily or hourly pattern? 

We start by looking at the autocorrelations, which is the correlation between the variable (grid frequency) and it's shifted amount (1 hour or 1 day or any time window; we use a range below). Note that for each hour, there is a lot happening; the data is at 1hz resolution. So we downsample to the hour by simply taking the mean or max. We lose a lot of resolution this way, but don't need it to get an understanding of these crude patterns.

In [ ]:
# Plot the autocorrelation of the 1-hour medians and maxs too look for a pattern
meds = nz.resample('1h').grid_freq_centered.median()
maxs = nz.resample('1h').grid_freq_centered.max()
med_corrs = [meds.autocorr(lag=i) for i in range(1, 73)]
max_corrs = [maxs.autocorr(lag=i) for i in range(1, 73)]
wdf = pd.DataFrame(data={'med': med_corrs, 'max': max_corrs}, index=np.arange(1, 73))
fig = px.line(wdf, labels=dict(index='hours', value='autocorrelation'))
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, 73, 12),
    )
)

We see a significant daily trend in the median and maxes. Unexpectedly there's also strong 12-hour trend; I'm not sure what causes that.

Because of the autocorrelations, it appears that the grid frequency is similar day-over-day. Let's look at individual hours and see if there's a trend there. Within an hour, is the grid frequency usually above where it needs to be (50k) or below? To measure this, we look at the *sign* of the centered grid frequencies and sum them.

In [186]:
wdf = bess.resample('1h').grid_freq_sign.sum().to_frame()
wdf['h'] = wdf.index.hour
fig = px.box(wdf, x='h', y='grid_freq_sign')
fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, 24),
    )
)

This reveals a pretty noticeable pattern. 
* From midnight to 7, the grid frequency is pretty close to 50k, generally a little under in the early morning.
* From 8-12 the grid frequency is usually high
* Then it generally decreases in the late afternoon, and sits below 50k until late in the evening (20-21)
* Finally at 20-21, it rises back above 50k until around midnight.

As for the 12 hour autocorrelation we saw above, we see that both early morning (4-7) and early afternoon (16-19) have low frequencies, whereas late morning and late evening have high frequencies. However, I suspect that the exact shift (12-hours) is essentially coincidental and not because it's half a day.

# When frequency is X, how long before it returns to 50k? Is this predictable?